# Time Series Analysis with SARPYX

This notebook demonstrates advanced time series analysis capabilities in SARPYX for monitoring ground deformation over time.

## Overview

Time series InSAR analysis enables:
- Long-term deformation monitoring
- Separation of different deformation signals
- Atmospheric artifact removal
- Persistent scatterer identification
- Trend and seasonal component analysis

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
import seaborn as sns
from scipy import signal, stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# SARPYX time series modules
from sarpyx.science.timeseries import (
    TimeSeriesAnalyzer,
    PersistentScattererProcessor,
    AtmosphericCorrector,
    DeformationModeler
)
from sarpyx.utils import setup_logging

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Set up logging
setup_logging(level='INFO')

print("✓ SARPYX Time Series modules imported successfully")

## Generate Synthetic Time Series Data

For demonstration, we'll create realistic synthetic InSAR time series data:

In [ ]:
# Generate time series parameters
start_date = datetime(2020, 1, 1)
end_date = datetime(2023, 12, 31)
acquisition_interval = 12  # days (Sentinel-1 repeat cycle)

# Generate acquisition dates
dates = []
current_date = start_date
while current_date <= end_date:
    dates.append(current_date)
    current_date += timedelta(days=acquisition_interval)

n_acquisitions = len(dates)
time_years = np.array([(d - start_date).days / 365.25 for d in dates])

print(f"Generated {n_acquisitions} acquisition dates from {start_date.date()} to {end_date.date()}")
print(f"Time span: {time_years[-1]:.1f} years")

# Scene parameters
scene_width, scene_height = 100, 80
n_pixels = scene_width * scene_height

print(f"Scene dimensions: {scene_width} x {scene_height} pixels ({n_pixels:,} total)")

### Create Realistic Deformation Patterns

In [ ]:
def generate_deformation_components(x, y, time_years):
    """
    Generate realistic deformation components:
    - Linear trend (tectonic/subsidence)
    - Seasonal variation (thermal expansion, hydrological)
    - Nonlinear trend (exponential decay, acceleration)
    - Episodic events (earthquakes, sudden changes)
    """
    n_times = len(time_years)
    
    # 1. Linear trend - varies spatially
    # Subsidence bowl in the center
    center_x, center_y = scene_width // 2, scene_height // 2
    distance_from_center = np.sqrt((x - center_x)**2 + (y - center_y)**2)
    max_subsidence_rate = -15  # mm/year at center
    
    linear_rate = max_subsidence_rate * np.exp(-distance_from_center**2 / (2 * 20**2))
    linear_component = np.outer(linear_rate, time_years)
    
    # 2. Seasonal variation
    seasonal_amplitude = 5.0 * np.exp(-distance_from_center**2 / (2 * 30**2))
    seasonal_component = np.outer(seasonal_amplitude, 
                                 np.sin(2 * np.pi * time_years + np.pi/4))
    
    # 3. Nonlinear component (exponential)
    nonlinear_amplitude = -3.0 * np.exp(-distance_from_center**2 / (2 * 15**2))
    nonlinear_component = np.outer(nonlinear_amplitude, 
                                  (1 - np.exp(-time_years / 2)))
    
    # 4. Episodic event (simulated earthquake at t=2.5 years)
    event_time = 2.5
    event_amplitude = -8.0 * np.exp(-distance_from_center**2 / (2 * 25**2))
    event_component = np.outer(event_amplitude, 
                              (time_years > event_time).astype(float))
    
    return linear_component, seasonal_component, nonlinear_component, event_component

# Generate coordinate grids
x_coords, y_coords = np.meshgrid(range(scene_width), range(scene_height))

# Generate deformation components for all pixels
print("Generating deformation components...")
linear_def, seasonal_def, nonlinear_def, event_def = generate_deformation_components(
    x_coords, y_coords, time_years
)

# Combine all components
true_deformation = linear_def + seasonal_def + nonlinear_def + event_def

print("✓ Deformation components generated")
print(f"  Linear trend range: {np.min(linear_def[:, -1]):.1f} to {np.max(linear_def[:, -1]):.1f} mm")
print(f"  Seasonal amplitude range: {np.min(seasonal_def):.1f} to {np.max(seasonal_def):.1f} mm")
print(f"  Event magnitude range: {np.min(event_def):.1f} to {np.max(event_def):.1f} mm")

### Add Atmospheric and Noise Components

In [ ]:
def generate_atmospheric_effects(scene_width, scene_height, n_times):
    """
    Generate realistic atmospheric phase delay patterns
    """
    # Stratified atmosphere (elevation-dependent)
    # Simulate topography
    x = np.linspace(0, 10, scene_width)
    y = np.linspace(0, 8, scene_height)
    X, Y = np.meshgrid(x, y)
    elevation = 200 + 300 * np.sin(np.pi * X / 10) * np.cos(np.pi * Y / 8)
    
    # Atmospheric delay coefficient (varies by acquisition)
    atm_coeffs = 0.5 + 1.5 * np.random.randn(n_times)  # mm per 100m elevation
    
    # Calculate stratified delays
    stratified_atm = np.zeros((scene_height, scene_width, n_times))
    for i in range(n_times):
        stratified_atm[:, :, i] = atm_coeffs[i] * elevation / 100
    
    # Turbulent atmosphere (spatial correlation)
    turbulent_atm = np.zeros((scene_height, scene_width, n_times))
    for i in range(n_times):
        # Generate correlated noise
        noise = np.random.randn(scene_height, scene_width)
        # Apply Gaussian filter for spatial correlation
        from scipy.ndimage import gaussian_filter
        correlated_noise = gaussian_filter(noise, sigma=5)
        turbulent_atm[:, :, i] = 3.0 * correlated_noise
    
    return stratified_atm, turbulent_atm

# Generate atmospheric effects
print("Generating atmospheric effects...")
stratified_atm, turbulent_atm = generate_atmospheric_effects(
    scene_width, scene_height, n_acquisitions
)

total_atmosphere = stratified_atm + turbulent_atm

# Add measurement noise
noise_std = 2.0  # mm standard deviation
measurement_noise = noise_std * np.random.randn(
    scene_height, scene_width, n_acquisitions
)

# Create observed time series (reshape for easier handling)
observed_ts = true_deformation + total_atmosphere + measurement_noise

print("✓ Atmospheric effects and noise added")
print(f"  Stratified atmosphere range: ±{np.std(stratified_atm):.1f} mm")
print(f"  Turbulent atmosphere range: ±{np.std(turbulent_atm):.1f} mm")
print(f"  Measurement noise: {noise_std:.1f} mm std")

## Visualize the Synthetic Dataset

In [ ]:
# Plot overview of the synthetic dataset
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Time indices for visualization
mid_time = n_acquisitions // 2
end_time = n_acquisitions - 1

# True deformation at different times
im1 = axes[0, 0].imshow(true_deformation[:, :, mid_time], cmap='RdBu_r')
axes[0, 0].set_title(f'True Deformation\n{dates[mid_time].strftime("%Y-%m-%d")}')
plt.colorbar(im1, ax=axes[0, 0], label='Displacement (mm)')

im2 = axes[0, 1].imshow(true_deformation[:, :, end_time], cmap='RdBu_r')
axes[0, 1].set_title(f'True Deformation\n{dates[end_time].strftime("%Y-%m-%d")}')
plt.colorbar(im2, ax=axes[0, 1], label='Displacement (mm)')

# Atmospheric effects
im3 = axes[0, 2].imshow(total_atmosphere[:, :, mid_time], cmap='RdBu_r')
axes[0, 2].set_title(f'Atmospheric Effects\n{dates[mid_time].strftime("%Y-%m-%d")}')
plt.colorbar(im3, ax=axes[0, 2], label='Atmospheric delay (mm)')

# Observed data
im4 = axes[1, 0].imshow(observed_ts[:, :, mid_time], cmap='RdBu_r')
axes[1, 0].set_title(f'Observed Data\n{dates[mid_time].strftime("%Y-%m-%d")}')
plt.colorbar(im4, ax=axes[1, 0], label='Displacement (mm)')

im5 = axes[1, 1].imshow(observed_ts[:, :, end_time], cmap='RdBu_r')
axes[1, 1].set_title(f'Observed Data\n{dates[end_time].strftime("%Y-%m-%d")}')
plt.colorbar(im5, ax=axes[1, 1], label='Displacement (mm)')

# Linear velocity map
velocity_map = (true_deformation[:, :, -1] - true_deformation[:, :, 0]) / time_years[-1]
im6 = axes[1, 2].imshow(velocity_map, cmap='RdBu_r')
axes[1, 2].set_title('True Linear Velocity')
plt.colorbar(im6, ax=axes[1, 2], label='Velocity (mm/year)')

# Remove axis ticks for cleaner look
for ax in axes.flatten():
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
plt.show()

print(f"Dataset overview:")
print(f"  Scene size: {scene_width} x {scene_height} pixels")
print(f"  Time series length: {n_acquisitions} acquisitions")
print(f"  Maximum deformation: {np.max(true_deformation):.1f} mm")
print(f"  Minimum deformation: {np.min(true_deformation):.1f} mm")
print(f"  Maximum velocity: {np.max(velocity_map):.1f} mm/year")
print(f"  Minimum velocity: {np.min(velocity_map):.1f} mm/year")

## Time Series Analysis: Point Examples

Let's examine time series at specific points of interest:

In [ ]:
# Define points of interest
poi_locations = [
    (scene_width//2, scene_height//2, "Subsidence Center"),
    (scene_width//4, scene_height//4, "Edge Region"),
    (3*scene_width//4, 3*scene_height//4, "Reference Area"),
    (scene_width//6, 5*scene_height//6, "Stable Region")
]

# Extract time series for each POI
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
axes = [ax1, ax2, ax3, ax4]
colors = ['red', 'blue', 'green', 'orange']

for i, (x, y, name) in enumerate(poi_locations):
    # Extract time series
    true_ts = true_deformation[y, x, :]
    observed_ts_point = observed_ts[y, x, :]
    atm_ts = total_atmosphere[y, x, :]
    
    # Plot on individual subplot
    ax = axes[i]
    ax.plot(dates, true_ts, 'k-', linewidth=2, label='True deformation')
    ax.plot(dates, observed_ts_point, 'o-', color=colors[i], 
           alpha=0.7, markersize=4, label='Observed')
    ax.plot(dates, atm_ts, '--', color='gray', alpha=0.7, label='Atmospheric')
    
    ax.set_title(f'{name}\nLocation: ({x}, {y})')
    ax.set_ylabel('Displacement (mm)')
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    # Format dates
    if i >= 2:  # Bottom row
        ax.tick_params(axis='x', rotation=45)
    else:
        ax.set_xticklabels([])

plt.suptitle('Time Series Examples at Different Locations', fontsize=16)
plt.tight_layout()
plt.show()

# Calculate and display statistics for each POI
print("Time Series Statistics:")
for x, y, name in poi_locations:
    true_ts = true_deformation[y, x, :]
    observed_ts_point = observed_ts[y, x, :]
    
    # Calculate linear velocity
    slope, intercept, r_value, p_value, std_err = stats.linregress(time_years, true_ts)
    
    # Calculate RMSE between true and observed
    rmse = np.sqrt(np.mean((true_ts - observed_ts_point)**2))
    
    print(f"  {name}:")
    print(f"    True velocity: {slope:.1f} ± {std_err:.1f} mm/year")
    print(f"    RMSE (obs vs true): {rmse:.1f} mm")
    print(f"    Total displacement: {true_ts[-1] - true_ts[0]:.1f} mm")

## Time Series Decomposition

Decompose the time series into trend, seasonal, and residual components:

In [ ]:
from scipy.signal import savgol_filter
from sklearn.linear_model import LinearRegression

def decompose_time_series(ts_data, time_years, window_length=25):
    """
    Decompose time series into trend, seasonal, and residual components
    """
    # 1. Extract linear trend
    X = time_years.reshape(-1, 1)
    reg = LinearRegression().fit(X, ts_data)
    linear_trend = reg.predict(X)
    
    # 2. Extract nonlinear trend using Savitzky-Golay filter
    if len(ts_data) > window_length:
        smooth_trend = savgol_filter(ts_data, window_length, 3)
    else:
        smooth_trend = linear_trend
    
    # 3. Remove trend to get seasonal + residual
    detrended = ts_data - smooth_trend
    
    # 4. Extract seasonal component (annual cycle)
    # Fit sinusoidal model
    def seasonal_model(t, a, b, c):
        return a * np.sin(2 * np.pi * t) + b * np.cos(2 * np.pi * t) + c
    
    from scipy.optimize import curve_fit
    try:
        popt, _ = curve_fit(seasonal_model, time_years, detrended)
        seasonal = seasonal_model(time_years, *popt)
    except:
        seasonal = np.zeros_like(time_years)
    
    # 5. Residual
    residual = ts_data - smooth_trend - seasonal
    
    return {
        'linear_trend': linear_trend,
        'smooth_trend': smooth_trend,
        'seasonal': seasonal,
        'residual': residual,
        'linear_velocity': reg.coef_[0]
    }

# Decompose time series for the subsidence center
center_x, center_y = scene_width//2, scene_height//2
center_ts = observed_ts[center_y, center_x, :]
decomposition = decompose_time_series(center_ts, time_years)

# Plot decomposition
fig, axes = plt.subplots(4, 1, figsize=(14, 12))

# Original time series
axes[0].plot(dates, center_ts, 'b-o', markersize=4, label='Observed')
axes[0].plot(dates, true_deformation[center_y, center_x, :], 'k--', 
            linewidth=2, label='True')
axes[0].set_ylabel('Displacement (mm)')
axes[0].set_title('Original Time Series (Subsidence Center)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Trend component
axes[1].plot(dates, decomposition['smooth_trend'], 'r-', linewidth=2, label='Smooth trend')
axes[1].plot(dates, decomposition['linear_trend'], 'r--', linewidth=2, label='Linear trend')
axes[1].set_ylabel('Displacement (mm)')
axes[1].set_title(f'Trend Component (Linear velocity: {decomposition["linear_velocity"]:.1f} mm/year)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Seasonal component
axes[2].plot(dates, decomposition['seasonal'], 'g-', linewidth=2)
axes[2].set_ylabel('Displacement (mm)')
axes[2].set_title('Seasonal Component')
axes[2].grid(True, alpha=0.3)

# Residual component
axes[3].plot(dates, decomposition['residual'], 'orange', alpha=0.7)
axes[3].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[3].set_ylabel('Displacement (mm)')
axes[3].set_xlabel('Date')
axes[3].set_title(f'Residual Component (σ = {np.std(decomposition["residual"]):.1f} mm)')
axes[3].grid(True, alpha=0.3)

# Format x-axis
for ax in axes:
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"Time Series Decomposition Results:")
print(f"  Linear velocity: {decomposition['linear_velocity']:.2f} mm/year")
print(f"  Seasonal amplitude: {np.std(decomposition['seasonal']):.1f} mm")
print(f"  Residual std: {np.std(decomposition['residual']):.1f} mm")
print(f"  Total variance explained: {1 - np.var(decomposition['residual'])/np.var(center_ts):.1%}")

## Spatial Pattern Analysis

Analyze spatial patterns in the time series using Principal Component Analysis (PCA):

In [ ]:
# Reshape data for PCA (pixels x time)
ts_matrix = observed_ts.reshape(-1, n_acquisitions)

# Remove mean (center the data)
ts_matrix_centered = ts_matrix - np.mean(ts_matrix, axis=1, keepdims=True)

# Apply PCA
n_components = 5
pca = PCA(n_components=n_components)
pca_scores = pca.fit_transform(ts_matrix_centered.T)
pca_components = pca.components_

# Calculate explained variance
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

print(f"PCA Analysis Results:")
for i in range(n_components):
    print(f"  Component {i+1}: {explained_variance[i]:.1%} variance")
print(f"  Total explained variance (first {n_components} components): {cumulative_variance[-1]:.1%}")

# Visualize PCA results
fig, axes = plt.subplots(3, 3, figsize=(18, 15))

# Plot explained variance
axes[0, 0].bar(range(1, n_components+1), explained_variance, alpha=0.7)
axes[0, 0].plot(range(1, n_components+1), cumulative_variance, 'ro-')
axes[0, 0].set_xlabel('Component')
axes[0, 0].set_ylabel('Explained Variance')
axes[0, 0].set_title('PCA Explained Variance')
axes[0, 0].grid(True, alpha=0.3)

# Plot temporal patterns (first 3 components)
for i in range(3):
    row, col = (0, 1) if i == 0 else (0, 2) if i == 1 else (1, 0)
    
    axes[row, col].plot(dates, pca_scores[:, i], 'o-', linewidth=2)
    axes[row, col].set_title(f'PC{i+1} Temporal Pattern\n({explained_variance[i]:.1%} variance)')
    axes[row, col].set_ylabel('PC Score')
    axes[row, col].grid(True, alpha=0.3)
    axes[row, col].tick_params(axis='x', rotation=45)

# Plot spatial patterns (first 3 components)
for i in range(3):
    row, col = (1, 1) if i == 0 else (1, 2) if i == 1 else (2, 0)
    
    spatial_pattern = pca_components[i, :].reshape(scene_height, scene_width)
    im = axes[row, col].imshow(spatial_pattern, cmap='RdBu_r')
    axes[row, col].set_title(f'PC{i+1} Spatial Pattern')
    axes[row, col].set_xticks([])
    axes[row, col].set_yticks([])
    plt.colorbar(im, ax=axes[row, col], shrink=0.8)

# Reconstruction quality
reconstructed = np.dot(pca_scores[:, :3], pca_components[:3, :]).T
reconstructed += np.mean(ts_matrix, axis=1, keepdims=True)

# Show reconstruction for center pixel
center_idx = center_y * scene_width + center_x
axes[2, 1].plot(dates, ts_matrix[center_idx, :], 'b-o', label='Original', alpha=0.7)
axes[2, 1].plot(dates, reconstructed[center_idx, :], 'r-', linewidth=2, label='PC1-3 reconstruction')
axes[2, 1].set_title('PCA Reconstruction\n(Center pixel)')
axes[2, 1].set_ylabel('Displacement (mm)')
axes[2, 1].legend()
axes[2, 1].grid(True, alpha=0.3)
axes[2, 1].tick_params(axis='x', rotation=45)

# Calculate reconstruction RMSE map
rmse_map = np.sqrt(np.mean((ts_matrix - reconstructed)**2, axis=1))
rmse_map = rmse_map.reshape(scene_height, scene_width)

im = axes[2, 2].imshow(rmse_map, cmap='viridis')
axes[2, 2].set_title('Reconstruction RMSE\n(3 components)')
axes[2, 2].set_xticks([])
axes[2, 2].set_yticks([])
plt.colorbar(im, ax=axes[2, 2], shrink=0.8, label='RMSE (mm)')

plt.tight_layout()
plt.show()

print(f"\nPCA Reconstruction Quality:")
print(f"  Mean RMSE: {np.mean(rmse_map):.1f} mm")
print(f"  Max RMSE: {np.max(rmse_map):.1f} mm")
print(f"  Reconstruction efficiency: {1 - np.mean(rmse_map**2)/np.var(ts_matrix):.1%}")

## Atmospheric Correction

Demonstrate atmospheric phase correction techniques:

In [ ]:
def atmospheric_correction_pca(ts_data, n_components_remove=2):
    """
    Remove atmospheric components using PCA
    Assumes first few PC components contain atmospheric signals
    """
    # Center the data
    ts_centered = ts_data - np.mean(ts_data, axis=2, keepdims=True)
    ts_matrix = ts_centered.reshape(-1, ts_centered.shape[2])
    
    # Apply PCA
    pca = PCA()
    pca_scores = pca.fit_transform(ts_matrix.T)
    pca_components = pca.components_
    
    # Remove first n components (assumed to be atmospheric)
    pca_scores_corrected = pca_scores.copy()
    pca_scores_corrected[:, :n_components_remove] = 0
    
    # Reconstruct corrected time series
    ts_corrected = np.dot(pca_scores_corrected, pca_components).T
    ts_corrected += np.mean(ts_data, axis=2, keepdims=True)
    
    return ts_corrected.reshape(ts_data.shape)

def atmospheric_correction_spatial_filter(ts_data, filter_size=10):
    """
    Remove atmospheric effects using spatial high-pass filtering
    """
    from scipy.ndimage import uniform_filter
    
    corrected_ts = np.zeros_like(ts_data)
    
    for t in range(ts_data.shape[2]):
        # Apply spatial low-pass filter to estimate atmospheric component
        atm_estimate = uniform_filter(ts_data[:, :, t], size=filter_size)
        
        # Subtract atmospheric estimate
        corrected_ts[:, :, t] = ts_data[:, :, t] - atm_estimate
    
    return corrected_ts

# Apply atmospheric corrections
print("Applying atmospheric corrections...")

# PCA-based correction
ts_corrected_pca = atmospheric_correction_pca(observed_ts, n_components_remove=2)

# Spatial filter correction
ts_corrected_spatial = atmospheric_correction_spatial_filter(observed_ts, filter_size=15)

# Compare correction methods
fig, axes = plt.subplots(2, 4, figsize=(20, 10))

# Time index for visualization
t_idx = n_acquisitions // 2

# Original data
im1 = axes[0, 0].imshow(observed_ts[:, :, t_idx], cmap='RdBu_r')
axes[0, 0].set_title('Original Observed')
plt.colorbar(im1, ax=axes[0, 0], shrink=0.8)

# PCA corrected
im2 = axes[0, 1].imshow(ts_corrected_pca[:, :, t_idx], cmap='RdBu_r')
axes[0, 1].set_title('PCA Corrected')
plt.colorbar(im2, ax=axes[0, 1], shrink=0.8)

# Spatial filter corrected
im3 = axes[0, 2].imshow(ts_corrected_spatial[:, :, t_idx], cmap='RdBu_r')
axes[0, 2].set_title('Spatial Filter Corrected')
plt.colorbar(im3, ax=axes[0, 2], shrink=0.8)

# True deformation (reference)
im4 = axes[0, 3].imshow(true_deformation[:, :, t_idx], cmap='RdBu_r')
axes[0, 3].set_title('True Deformation')
plt.colorbar(im4, ax=axes[0, 3], shrink=0.8)

# Time series comparison at center pixel
center_original = observed_ts[center_y, center_x, :]
center_pca = ts_corrected_pca[center_y, center_x, :]
center_spatial = ts_corrected_spatial[center_y, center_x, :]
center_true = true_deformation[center_y, center_x, :]

axes[1, 0].plot(dates, center_original, 'b-o', alpha=0.7, label='Original')
axes[1, 0].plot(dates, center_true, 'k--', linewidth=2, label='True')
axes[1, 0].set_title('Original vs True')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].tick_params(axis='x', rotation=45)

axes[1, 1].plot(dates, center_pca, 'r-o', alpha=0.7, label='PCA corrected')
axes[1, 1].plot(dates, center_true, 'k--', linewidth=2, label='True')
axes[1, 1].set_title('PCA Corrected vs True')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].tick_params(axis='x', rotation=45)

axes[1, 2].plot(dates, center_spatial, 'g-o', alpha=0.7, label='Spatial corrected')
axes[1, 2].plot(dates, center_true, 'k--', linewidth=2, label='True')
axes[1, 2].set_title('Spatial Corrected vs True')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)
axes[1, 2].tick_params(axis='x', rotation=45)

# RMSE comparison
rmse_original = np.sqrt(np.mean((observed_ts - true_deformation)**2))
rmse_pca = np.sqrt(np.mean((ts_corrected_pca - true_deformation)**2))
rmse_spatial = np.sqrt(np.mean((ts_corrected_spatial - true_deformation)**2))

methods = ['Original', 'PCA Corrected', 'Spatial Corrected']
rmse_values = [rmse_original, rmse_pca, rmse_spatial]

bars = axes[1, 3].bar(methods, rmse_values, color=['blue', 'red', 'green'], alpha=0.7)
axes[1, 3].set_ylabel('RMSE (mm)')
axes[1, 3].set_title('Correction Performance')
axes[1, 3].grid(True, alpha=0.3)

# Add value labels on bars
for bar, value in zip(bars, rmse_values):
    axes[1, 3].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                   f'{value:.1f}', ha='center', va='bottom')

# Remove ticks for image plots
for i in range(4):
    axes[0, i].set_xticks([])
    axes[0, i].set_yticks([])

plt.tight_layout()
plt.show()

print(f"Atmospheric Correction Results:")
print(f"  Original RMSE: {rmse_original:.1f} mm")
print(f"  PCA corrected RMSE: {rmse_pca:.1f} mm ({(rmse_original-rmse_pca)/rmse_original*100:+.1f}%)")
print(f"  Spatial corrected RMSE: {rmse_spatial:.1f} mm ({(rmse_original-rmse_spatial)/rmse_original*100:+.1f}%)")

# Best performing method
best_method_idx = np.argmin(rmse_values[1:])
best_method = methods[best_method_idx + 1]
print(f"  Best performing method: {best_method}")

## Velocity Estimation and Uncertainty

Estimate linear velocities and their uncertainties:

In [ ]:
def estimate_velocity_map(ts_data, time_years, method='linear'):
    """
    Estimate velocity map from time series data
    """
    height, width, n_times = ts_data.shape
    
    velocity_map = np.zeros((height, width))
    velocity_std_map = np.zeros((height, width))
    r_squared_map = np.zeros((height, width))
    
    X = time_years.reshape(-1, 1)
    
    for i in range(height):
        for j in range(width):
            ts = ts_data[i, j, :]
            
            # Linear regression
            slope, intercept, r_value, p_value, std_err = stats.linregress(time_years, ts)
            
            velocity_map[i, j] = slope
            velocity_std_map[i, j] = std_err
            r_squared_map[i, j] = r_value**2
    
    return velocity_map, velocity_std_map, r_squared_map

# Estimate velocities for different datasets
print("Estimating velocity maps...")

# True velocities
vel_true, vel_std_true, r2_true = estimate_velocity_map(true_deformation, time_years)

# Observed velocities (with atmospheric effects)
vel_observed, vel_std_observed, r2_observed = estimate_velocity_map(observed_ts, time_years)

# Corrected velocities (using best correction method)
if rmse_pca < rmse_spatial:
    vel_corrected, vel_std_corrected, r2_corrected = estimate_velocity_map(ts_corrected_pca, time_years)
    correction_label = "PCA Corrected"
else:
    vel_corrected, vel_std_corrected, r2_corrected = estimate_velocity_map(ts_corrected_spatial, time_years)
    correction_label = "Spatial Corrected"

# Visualize velocity maps
fig, axes = plt.subplots(2, 4, figsize=(20, 10))

# Velocity maps
vmin, vmax = np.percentile(vel_true, [5, 95])

im1 = axes[0, 0].imshow(vel_true, cmap='RdBu_r', vmin=vmin, vmax=vmax)
axes[0, 0].set_title('True Velocity')
plt.colorbar(im1, ax=axes[0, 0], shrink=0.8, label='mm/year')

im2 = axes[0, 1].imshow(vel_observed, cmap='RdBu_r', vmin=vmin, vmax=vmax)
axes[0, 1].set_title('Observed Velocity')
plt.colorbar(im2, ax=axes[0, 1], shrink=0.8, label='mm/year')

im3 = axes[0, 2].imshow(vel_corrected, cmap='RdBu_r', vmin=vmin, vmax=vmax)
axes[0, 2].set_title(f'{correction_label} Velocity')
plt.colorbar(im3, ax=axes[0, 2], shrink=0.8, label='mm/year')

# Velocity difference
vel_diff = vel_corrected - vel_true
im4 = axes[0, 3].imshow(vel_diff, cmap='RdBu_r')
axes[0, 3].set_title('Velocity Difference\n(Corrected - True)')
plt.colorbar(im4, ax=axes[0, 3], shrink=0.8, label='mm/year')

# Uncertainty maps
im5 = axes[1, 0].imshow(vel_std_true, cmap='viridis')
axes[1, 0].set_title('True Velocity Uncertainty')
plt.colorbar(im5, ax=axes[1, 0], shrink=0.8, label='mm/year')

im6 = axes[1, 1].imshow(vel_std_observed, cmap='viridis')
axes[1, 1].set_title('Observed Velocity Uncertainty')
plt.colorbar(im6, ax=axes[1, 1], shrink=0.8, label='mm/year')

im7 = axes[1, 2].imshow(r2_corrected, cmap='viridis', vmin=0, vmax=1)
axes[1, 2].set_title('R² (Goodness of Fit)')
plt.colorbar(im7, ax=axes[1, 2], shrink=0.8, label='R²')

# Scatter plot: True vs Corrected velocities
# Sample for plotting
sample_indices = np.random.choice(n_pixels, 1000, replace=False)
vel_true_flat = vel_true.flatten()[sample_indices]
vel_corrected_flat = vel_corrected.flatten()[sample_indices]

axes[1, 3].scatter(vel_true_flat, vel_corrected_flat, alpha=0.6, s=20)
axes[1, 3].plot([vmin, vmax], [vmin, vmax], 'r--', linewidth=2, label='1:1 line')
axes[1, 3].set_xlabel('True Velocity (mm/year)')
axes[1, 3].set_ylabel('Corrected Velocity (mm/year)')
axes[1, 3].set_title('Velocity Comparison')
axes[1, 3].legend()
axes[1, 3].grid(True, alpha=0.3)

# Calculate correlation
correlation = np.corrcoef(vel_true.flatten(), vel_corrected.flatten())[0, 1]
axes[1, 3].text(0.05, 0.95, f'Correlation: {correlation:.3f}', 
               transform=axes[1, 3].transAxes, 
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Remove ticks for image plots
for i in range(3):
    for j in range(4):
        if j < 3:  # Skip scatter plot
            axes[i, j].set_xticks([])
            axes[i, j].set_yticks([])

plt.tight_layout()
plt.show()

# Calculate velocity statistics
vel_rmse = np.sqrt(np.mean((vel_corrected - vel_true)**2))
vel_bias = np.mean(vel_corrected - vel_true)
vel_std_diff = np.std(vel_corrected - vel_true)

print(f"\nVelocity Estimation Results:")
print(f"  True velocity range: {np.min(vel_true):.1f} to {np.max(vel_true):.1f} mm/year")
print(f"  Velocity RMSE: {vel_rmse:.2f} mm/year")
print(f"  Velocity bias: {vel_bias:.2f} mm/year")
print(f"  Velocity correlation: {correlation:.3f}")
print(f"  Mean R²: {np.mean(r2_corrected):.3f}")
print(f"  Pixels with R² > 0.8: {np.sum(r2_corrected > 0.8)/n_pixels*100:.1f}%")

## Summary and Best Practices

This notebook demonstrated comprehensive time series analysis techniques for InSAR data:

### Key Techniques Covered:
1. **Time Series Decomposition**: Separating trend, seasonal, and residual components
2. **Spatial Pattern Analysis**: Using PCA to identify common spatial patterns
3. **Atmospheric Correction**: PCA-based and spatial filtering approaches
4. **Velocity Estimation**: Linear trend estimation with uncertainty quantification
5. **Quality Assessment**: R² and correlation analysis

### Best Practices for Time Series InSAR:

1. **Data Quality**:
   - Use consistent acquisition geometry
   - Maintain regular temporal sampling
   - Consider coherence thresholds

2. **Atmospheric Correction**:
   - Apply spatial filtering for stratified delays
   - Use PCA for common mode removal
   - Consider external weather data

3. **Trend Analysis**:
   - Account for nonlinear deformation
   - Consider seasonal variations
   - Validate with independent data

4. **Uncertainty Quantification**:
   - Estimate velocity uncertainties
   - Consider temporal correlation
   - Assess model goodness of fit

### Performance Summary:
- Atmospheric effects were successfully reduced by **{:.1f}%** using correction techniques
- Velocity estimation achieved **{:.1f} mm/year** RMSE
- **{:.1f}%** of pixels showed good linear fit (R² > 0.8)

## Next Steps

- Explore **Advanced Workflows** in `05_advanced_workflows.ipynb`
- Learn about **Multi-temporal InSAR** techniques
- Study **Persistent Scatterer** methods
- Investigate **Machine Learning** applications

---

*For more advanced time series techniques and applications, consult the SARPYX documentation and recent scientific literature.*